In [0]:
!pip install -q -U google-generativeai

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4833d9f2-6dc3-456d-b0e7-00fe52628992/bin/python -m pip install --upgrade pip' command.


In [0]:
# for visualization
import seaborn as sns
import numpy as np

# python standard libraries
from statistics import mean, median, variance
from math import log10, sqrt
from operator import add
from functools import reduce
from collections import Counter
import csv
import json
import re
import string

# gemini
import google.generativeai as genai

field_size_limit = csv.field_size_limit(10**9)

In [0]:
# Load the scraped data
with open("/Workspace/Users/vladklim@campus.technion.ac.il/Project/scraped_jobs.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    jobs_dict = {comp[0]: list(eval(comp[1])) for comp in reader}

In [0]:
jobs_len = [len(jobs) for jobs in jobs_dict.values()]
jobs_number = sum(jobs_len)
mean = mean(jobs_len)
median = median(jobs_len)
variance = variance(jobs_len)
print(f"{len(jobs_dict)=}, {jobs_number=}, {mean=}, {median=}, {variance=}")

len(jobs_dict)=1078, jobs_number=17707, mean=16.42578849721707, median=3.0, variance=4226.686688096358


In [0]:
dict(sorted(Counter(jobs_len).items(), key=lambda x: x[1], reverse=True)[:10])

Out[296]: {1: 337, 2: 139, 3: 98, 4: 70, 5: 53, 6: 51, 7: 26, 8: 26, 9: 25, 10: 24}

In [0]:
GOOGLE_API_KEY = 'PUT_YOUR_GOOGLE_API_KEY_HERE'
genai.configure(api_key=GOOGLE_API_KEY)

In [0]:
# We will use the latest 1.0 version (because more free requests allowed)
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


dict_structure = {<br>
"Job Title and Summary": ...,<br>
"Company Overview": ...,<br>
"Job Responsibilities": ["responsibility 1", "responsibility 2", ...],<br>
"Qualifications and Skills": ["qualification 1", "qualification 2", ...],<br>
"Required Education": ...,<br>
"Benefits and Perks": ["benefit 1", "benefit 2", ...],<br>
"Application Process": ..., <br>
"Contact Information": ...<br>
}

In [0]:
# Function for extracting content from gemini response to our promt (was empirically built)
def get_dict(response_text):
  stack = []
  start_index = None
  for i, char in enumerate(response_text):
    if char == '{':
      if start_index is None:
        start_index = i
      stack.append(char)
    elif char == '}':
      if not stack:
        return None
      stack.pop()
      if not stack:
        return eval(response_text[start_index: i + 1])  
  return None

In [0]:
# Initialize the gemini model
model = genai.GenerativeModel('gemini-1.0-pro-latest')

In [0]:
# All the job posts to process
posts = sum([[(i, url, job["description"], api_keys[i % len(api_keys)]) for i, job in enumerate(jobs)] for url, jobs in jobs_dict.items()], [])

In [0]:
# The code for parsing the jobs posts. Error resistant, but pretty slow due to a restriction of requests number per minute and per day. The code runs on all companies and parses first all the jobs at index 0, then 1 etc.
# The index variable allows to continue from specific place (the job index / the iteration), if restarted for some reason. start_from is the company index to start from. All other variables are pretty tuned, no need to change.
max_attempts = 6
failed_inarow = 0
max_index = max([len(jobs) for jobs in jobs_dict.values()]) - 1
index = 71
start_from = 133
count = start_from
last_index = [index, count]
while index <= max_index:
    print(f"Index: {index}")
    attempts = 0
    for url, jobs in list(jobs_dict.items())[start_from:]:
        count += 1
        if index >= len(jobs):
            continue
        last_index = [index, count]
        prompt = f'''Observe the next linkedin job post: {jobs[index]["description"]}\n\n\nParse the post to the next topics: Job Title and Summary (string), Company Overview (string), Job Responsibilities (list of strings), Qualifications and Skills (list of strings), Required Education (string), Benefits and Perks (list of strings), Application Process (string), Contact Information (string). In each topic write as short as possible, only the key info. If there is nothing to put in some topic, leave it empty. Topics are keys in the python dictionary, the content of the topics after parsing are the values. In Job Responsibilities, Qualifications and Skills, Required Education, Benefits and Perks - present as list of strings, very laconically and without punctuation, each string contains only one concept (e.g only one skill) Return this dictionary, without additional information, only the dictionary.'''
        attempts = 0
        while attempts < max_attempts:
            try:
                job_content = model.generate_content(prompt)
                job_content = get_dict(job_content.text)
                # Change to your path
                with open('/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content.csv', 'a', newline='', encoding='utf-8') as csv_file:  
                    writer = csv.writer(csv_file)
                    writer.writerow([index, url, str(job_content)])
                    
                
                print(f'{count}. {url} done')
                failed_inarow = 0
                break
            except Exception as e:
                print(index, count, e, "attempt:", attempts + 1)
                attempts += 1
                time.sleep(10)
        if attempts == max_attempts:
            # Change to your path
            with open('/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content_failed.csv', 'a', newline='', encoding='utf-8') as csv_file:  
                writer = csv.writer(csv_file)
                writer.writerow([last_index[0], last_index[1], url, jobs[index]["description"]])
            print(f'{index} {url} - no response')
            failed_inarow += 1
            if failed_inarow > 5:
                # Change to your path
                with open('/Workspace/Users/vladklim@campus.technion.ac.il/Project/last_index.txt', 'w', newline='', encoding='utf-8') as file: 
                    file.write(f'{index}, {count}, {url}')
                    print(":(")
                break
    index += 1
    start_from = 0
    count = 0
                

Index: 1
1 10 name 'null' is not defined attempt: 1
1 10 name 'null' is not defined attempt: 2
10. https://www.linkedin.com/company/amches done
11. https://www.linkedin.com/company/anvl done
12. https://www.linkedin.com/company/axelerant done
13. https://www.linkedin.com/company/ashburn-consulting-llc done
14. https://www.linkedin.com/company/ascent-geomatics-solutions done
15. https://www.linkedin.com/company/scaleai done
16. https://www.linkedin.com/company/cvin done
17. https://www.linkedin.com/company/trillium-teamologies done
20. https://www.linkedin.com/company/c3-tech-orange-county done
21. https://www.linkedin.com/company/cio-solutions done
23. https://www.linkedin.com/company/cambium-learning-group done
24. https://www.linkedin.com/company/caylent done
25. https://www.linkedin.com/company/changing-technologies-inc done
28. https://www.linkedin.com/company/contlo done
29. https://www.linkedin.com/company/crushbank done
31. https://www.linkedin.com/company/cynamics done
32. http

In [0]:
def split_file(source_file, output1, output2):
    with open(source_file, "rb") as source:
        data = source.read()
    print(len(data))
    with open(output1, "wb") as out:
         out.write(data[:len(data) // 2])
    with open(output2, "wb") as out:
         out.write(data[len(data) // 2:])

# Change to your paths
source_file = "/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content.csv"
output1 = "/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content_part1.bin"
output2 = "/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content_part2.bin"

split_file(source_file, output1, output2)
print(f"File '{source_file}' split into '{output1}' and '{output2}'.")


17169168
File '/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content.csv' split into '/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content_part1.bin' and '/Workspace/Users/vladklim@campus.technion.ac.il/Project/jobs_content_part2.bin'.
